## Coursera - IBM Data Science Capstone Project ##

### Brian Lee ###
BrianTrewLee@gmail.com<br />
LinkedIn: https://www.linkedin.com/in/brian-lee-64a2022b/



## 1. Introduction ##

If I were to open a Starbucks, where should I locate it? Clearly, the location is one of the most important business decisions for this venture. Is there a science to locating a StarBucks? Can we create a machine learning model to predict where a StarBucks is likely to be located? I'd like to find out.

If a reliable model can be made,it could be used in the process of opening a store. It could be used as a final sanity check, or at the beginning stage, to select a promising location. If a trustworthy model predicts with a high degree of confidence that a Starbucks should be located in an area, but there is not one there yet, perhaps that is an opportunity.

Any person interested in opening a Starbucks should be interested in these results. This includes myself, other potential franchisees, and the Starbucks corporation itself, which operates many of it's own stores. The Starbucks competition might also be interested as they could possibly gain competitive insigths. I also believe others might be interested in this procedure, as it might be applied to predicting the location of other entities, based on the same sort of data.

The main purpose of this project is to prove the concept of predicting Starbuck's locations in general. I suspect it may work better or worse depending on the locations chosen for training and prediction, the radius size, and the specific features used in the model. I may vary those factors in order to prove the concept, which could then be applied carefully to a particular geographic area of interest at a later date.

### 2. Data ###

Foursquare (https://foursquare.com) is a company that crowdsources location data, tying latitude-longitude coordinates (and other things) with public venues, including many businesses, such as Starbucks locations. The features for the Starbucks location classifier will come from venue data from the Fourquare API. I will use venue category names (such as 'bar', 'Chinese restaurant', 'coffee shop') as features to classify an area as either an area with a Starbucks, or an area without one. I will use the venue name (e.g. 'Starbucks') to determine whether or not the venue is a Starbucks. An area containing a Starbucks contains at least one venue named 'Starbucks' within the radius supplied to the Foursquare API's explore' endpoint.

An area will be a circular area with a given radius. I have not determined that radius yet, and I might use more than one radius in this project. Within that radius, the Foursquare API will return venues. The larger the radius, the more venues that could be anticipated, and the higher likelihood of a Starbucks, but a larger radius is also less useful for business use. I have done some exploratory work with a radius of 300m.

The areas to be chosen should contain a mix of areas with Starbuck's and those without. I have done some exploratory work in a particular area of San Francisco. I located Starbucks with Google Maps and got latitude and longitude points making a box around that area. I could choose the train and test points at random within an area, or I could generate a grid covering an entire area. Given the constraints my Foursquare Developer account places on my searches, and knowing that binary classifiers work better with a good number of examples of both classes, I may want to generate areas that I know will be heavy with Starbucks locations. To that end, I discovered a Kaggle dataset (https://www.kaggle.com/starbucks/store-locations) containing the latitude and longitude coordinates of 25,600 worldwide Starbucks locations. I may or may not use that resource.

### 3. Methodology ###

I will generate a corpus of geographic points, and using a reasonable radius (perhaps 300m) a corpus of geographic areas. Each geographic area will have a feature set based on json data returned by the Foursquare API's 'explore' endpoint. I plan to use the category names of the returned venues as features for that area. I will use the venue names to determine whether oir not a Starbucks is in that area, and generate the labels from that. 

Once I have labelled data, I will use sklearn.model_selection.train_test_split to generate tarining and testing sets. Then I  can use any of a host of sklearn classifiers to generate predictive models. I have done some exploratory work with a random forest.

In [2]:
# import libraries

import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

Libraries imported!


In [3]:
eliminate_starbucks_venue_rows = False

In [4]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
    
    count = 0
    venues_list = []
    for lat, lng in zip(latitudes, longitudes):
        #create a unique namefor this pointthat is easier todeal with than a combination of lat and lon
        point_name = "point"+str(count)
        print(point_name,':',lat,'-',lng)
         

            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        #print("resultscols=",results.columns)
        

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            point_name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["location"]["distance"], 
            v["venue"]["categories"][0]["name"]) for v in results])
        count = count + 1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Point Name",
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Distance",           
                  "Venue Category"]
    
    return(nearby_venues)

In [5]:
sf_lat=37.7749
sf_lon=-122.4194

import random

#37.780253, -122.509117
#37.804203, -122.459264 - upper left of starbucks intensive space
#37.767547, -122.409342 - lower right ofstarbucksintensive space
llat=37.767547
ulat=37.804203
llon=-122.459264
ulon=-122.409342

In [6]:
#37.599350, -122.496830
#37.807957, -122.390854
#reset parameters to bigger area
#llat = 37.599350
ulat = 37.807957
llon = -122.496830
##ulon = -122.390854

#new lower lat
llat=37.7
#new llon
#llon = -122.514508
llon = -122.494329
#new ulon
#ulon = -122.371542
ulon = -122.395276

points = []
radius = 300 #in meters
current_lat = ulat
current_lon = llon
import geopy
from geopy.distance import VincentyDistance
d = radius/1000 #in kilometers
d=d*2
b=90 #in degrees 90 = east,180=south,270=west
# given: lat1, lon1, b = bearing in degrees, d = distance in kilometers

def lon_pass(current_lat, current_lon, ulat, ulon):
    #while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat) and (current_lon >= llon):
    while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat):    
        origin = geopy.Point(current_lat, current_lon)
        destination = VincentyDistance(kilometers=d).destination(origin, b)

        current_lat, current_lon = destination.latitude, destination.longitude
        print(current_lat, current_lon)
        points.append((current_lat, current_lon))

def generate_points(llat, llon, ulat, ulon):
    print("1")
    current_lat = ulat
    current_lon = llon
    print(current_lat <= ulat)
    print(current_lon <= ulon)
    print(current_lat >= llat) 
    print("2")    
    #while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat) and (current_lon >= llon):
    while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat):
        print(current_lat <= ulat)
        print(current_lon <= ulon)
        print(current_lat >= llat)
        lon_pass(current_lat, current_lon, ulat, ulon)
        print('new row of coordinates')  
        current_lon=llon#move back to beginning of row   
        print('3')
        origin = geopy.Point(current_lat, current_lon)
        destination = VincentyDistance(kilometers=d).destination(origin, 180)#move south one step for next row
        current_lat, current_lon = destination.latitude, destination.longitude
        print('4')
        print(current_lat <= ulat)
        print(current_lon <= ulon)
        print(current_lat >= llat)  
        print("current_lat >= llat?")        
        print("current lat="+str(current_lat))
        print("llat="+str(llat))        

In [7]:
generate_points(llat, llon, ulat, ulon)

1
True
True
True
2
True
True
True
37.80795680296507 -122.48751554492938
37.807956605930144 -122.48070208987684
37.807956408895215 -122.47388863484242
37.80795621186029 -122.4670751798261
37.807956014825365 -122.46026172482789
37.80795581779044 -122.45344826984777
37.80795562075552 -122.44663481488575
37.80795542372061 -122.43982135994185
37.80795522668569 -122.43300790501604
37.80795502965077 -122.42619445010834
37.80795483261586 -122.41938099521875
37.80795463558094 -122.41256754034725
37.807954438546034 -122.40575408549387
37.807954241511126 -122.39894063065857
37.80795404447622 -122.3921271758414
new row of coordinates
3
4
True
True
True
current_lat >= llat?
current lat=37.8025512422552
llat=37.7
True
True
True
37.80255104525841 -122.48751604155096
37.80255084826162 -122.48070308312
37.80255065126483 -122.47389012470715
37.80255045426805 -122.46707716631241
37.802550257271264 -122.46026420793575
37.802550060274484 -122.45345124957718
37.802549863277704 -122.44663829123671
37.8025496

#### We many need more examples of the near-Starbucks class for a balanced set - so get a bunch of points we know are near to Starbucks
I pulled coordinates from the Kaggle Starbucks locations dataset<br />
The precision of the Kaggle set wasn't good enough to ensure the Starbucks location would be within the radius chosen, so I found a different dataset with higher precision

In [8]:
#starbucks_heavy_coordinates = pd.read_csv('data/directory-us-ca.csv') #this file gave lat-lon tothe hundreths, but that isn't good enough to ensure a Starbucks is within my radius
starbucks_heavy_coordinates = pd.read_csv('data/starbucks_us_ca_locations.csv',header=None)
starbucks_heavy_coordinates.columns = ["Longitude", "Latitude", "Location", "Address"]


In [ ]:
starbucks_heavy_coordinates.head()

,Longitude,Latitude,Location,Address
0,-118.114954,34.079000,Starbucks - CA - ALHAMBRA [W] 00373,"Valley & Almansor, Alhambra_810 E VALLEY BLVD_..."
1,-117.428300,34.626700,Starbucks - CA - Adelanto [WD] 00374,"Palmdale & Hwy 395, Adelanto_14136 US Hwy 395_..."
2,-118.757319,34.153435,Starbucks - CA - Agoura Hills 00375,Vons-Agoura Hills #2001_5671 Kanan Rd._Agoura ...
3,-118.757134,34.160600,Starbucks - CA - Agoura [W] 00376,"Agoura_5827 Canan Road_Agoura, California 9130..."
4,-122.277604,37.788222,Starbucks - CA - Alameda 00377,Albertsons-Alameda #7031_815 Marina Village_Al...


In [ ]:
#num_needed_to_balance = int(len(points)*1.5) #This balances the classes better
num_needed_to_balance = len(points) #This does a good enough job of balancing the classes
num_needed_to_balance

300

In [ ]:
shc_lat, shc_lon = starbucks_heavy_coordinates.Latitude, starbucks_heavy_coordinates.Longitude

nnl=(shc_lat[0:num_needed_to_balance],shc_lon[0:num_needed_to_balance])
pnnl = pd.DataFrame(nnl)
pnnl.shape
pnnl = pnnl.transpose()
pnnl
newlats=pnnl['Latitude']
newlons=pnnl['Longitude']
newlats
points = pd.DataFrame(points)
current_end = len(points)
for j in range(0,num_needed_to_balance):
    lenp = len(points)
    points = points.set_value(lenp, 0, newlats[j])
    points = points.set_value(lenp, 1, newlons[j])  

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [ ]:
points.shape

(600, 2)

In [ ]:
type(points)

pandas.core.frame.DataFrame

shuffle the points

In [ ]:
points = points.sample(frac=1).reset_index(drop=True)
points.head()

,0,1
0,37.807956,-122.446635
1,37.807955,-122.412568
2,34.160555,-118.345033
3,37.802551,-122.480703
4,37.732274,-122.412651


In [ ]:
points.tail()

,0,1
595,33.788253,-116.482265
596,37.786333,-122.453460
597,37.519696,-122.275314
598,37.721462,-122.405857
599,35.381182,-119.015281


In [ ]:
map = folium.Map(location=[ulat, ulon], zoom_start=11)


# add markers to map
for lat, lng in zip(points[0],points[1]):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

In [ ]:
venues = getNearbyVenues(latitudes = points[0],
                              longitudes =  points[1]
                               )


In [ ]:
venues.tail()

In [ ]:
venues['Venue'].unique()
venues['Venue'].value_counts()

In [ ]:
venues['Venue Category'].unique()

In [ ]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['point_name'] = venues['Point Name'] 
onehot['point_lat'] = venues['Neighborhood Latitude'] 
onehot['point_lon'] = venues['Neighborhood Longitude'] 
onehot['isStarbucks'] = (venues['Venue'] == "Starbucks")
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-3]] + [onehot.columns[-2]] + [onehot.columns[-1]] + list(onehot.columns[:-3])
onehot = onehot[fixed_columns]

onehot.isStarbucks = onehot.isStarbucks.astype(int)

onehot.head()
onehot['isStarbucks'].value_counts()

In [ ]:
venues.head()

In [ ]:
len(onehot)

In [ ]:
points_near_starbucks = onehot[onehot.isStarbucks > 0]["point_name"]
if eliminate_starbucks_venue_rows == True:
    #keep the isStarbucks flag but throw out other venue info
    ##points_near_starbucks = onehot[onehot.isStarbucks > 0]["point_name"]
    indexes_to_remember = onehot[onehot.isStarbucks > 0].index
    points_near_starbucks

In [ ]:
points_near_starbucks_df = pd.DataFrame(points_near_starbucks)
points_near_starbucks_df.head()

for point_name in points_near_starbucks_df.point_name:
    onehot.loc[onehot.point_name == point_name,"isStarbucks"] = 1  

In [ ]:
len(onehot)

In [ ]:
if eliminate_starbucks_venue_rows == True:
    for ind in indexes_to_remember:
        onehot = onehot.drop([ind], axis=0)

In [ ]:
len(onehot)

In [ ]:
len(onehot.columns)

In [ ]:
onehot['number_of_venues_within_radius'] = 1

In [ ]:
len(onehot.columns)
#onehot

My intuition is that an area with more venues is more business-centric, <br />
and that would affect the likelihood of a Starbucks being located nearby<br />
Let's create a feature for the number of venues nearby

In [ ]:
# create the new feature 'number_of_venues_within_radius'
# have to combine means and counts
grouper = onehot.groupby('point_name').mean().reset_index()
counter = onehot.groupby('point_name').count().reset_index()
grouped = grouper
grouped.iloc[:,-1] = counter.iloc[:,-1]
max_of_venue_count = grouped.iloc[:,-1].max()
# normalize the new feature
grouped.iloc[:,-1] = grouped.iloc[:,-1]/max_of_venue_count
grouped.iloc[:,-1].describe()

In [ ]:
verbose = False
num_top_venues = 5
count = 0
for point in grouped['point_name']:
    temp = grouped[grouped['point_name'] == point].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    if verbose == True:
        print("----"+point+"----")
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')
    else:
        if count % 100 == 0:
            print("----"+point+"----")
            print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
            print('\n')
    count = count + 1

In [ ]:
grouped.head()

In [ ]:
y = grouped['isStarbucks'] > 0
y = pd.DataFrame(y)
y = y.isStarbucks.astype(int)
print(y.value_counts())
X = grouped
print(X.shape)
X.drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)


In [ ]:
X.columns
#len(X.columns)

Create a random feature<br />
Let's see how many of our features do no better than a randomly generated feature

In [ ]:
X['random'] = np.random.random(size=len(X))

In [ ]:
print(X.columns)
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#X_train['random'] = np.random.random(size=len(X_train))
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
pdytrain=pd.DataFrame(y_train)
pdytest=pd.DataFrame(y_test)

pdytrain['isStarbucks'].value_counts()


In [ ]:
pdytest['isStarbucks'].value_counts()

In [ ]:
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier

#RF = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)  
##RF = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=0)
RF = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
fit = RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)
print("Train accuracy = " + str(RF.score(X_train,y_train)))

num_features = fit.n_features_
print("Num Features: " + str(num_features))  
feature_importances = RF.feature_importances_


In [ ]:
#feature_importances

In [ ]:
feature_importance_info = []
for i in range(0,num_features):
    if feature_importances[i] >= 0:
        feature_importance_info.append((feature_importances[i],X.columns[i]))

In [ ]:
pdfid = pd.DataFrame(feature_importance_info)
pdfid.rename(columns={0: "Feature Importance", 1: "Feature"}, inplace=True)
pdfid.sort_values(by="Feature Importance",ascending=False,inplace=True)

In [ ]:
pdfid.set_index('Feature',inplace=True)
pdfid

In [ ]:
corr = X.corr()
import seaborn as sns
sns.heatmap(corr)

Features are predominantly uncorrelated

In [ ]:
c = X.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")

In [ ]:
c.head()

In [ ]:
strong_correlations = []
correlation_threshold = 0.7
for colname in c.columns:
    print("check for features strongly coorelated with " + colname)
    for i in range(0, len(c)):
        this_correlation = c[colname][i]
        if this_correlation >= correlation_threshold:
            if colname != c.index[i]: #don't act on self correlations
                print("   " + c.index[i] + " - " + str(this_correlation))
                strong_correlations.append((colname,c.index[i],this_correlation))

Whether or not I do anything about correlated features, they are interesting to look at.<br />
Some of them appear to be negatively correlated even I only show the magnitude of the correlation here.<br />
Some of the correlations look like they are due to small samples, such as the Roller Rink in San Francisco that is perfectly correlated with a Spiritual Center. I've actually skated there.<br />
Some correlations look meaningful, such as the correlation between aquariums and zoos.


In [ ]:
strong_correlations_df = pd.DataFrame(strong_correlations)

In [ ]:
strong_correlations_df["feature_pair"]=strong_correlations_df[0]+"-"+strong_correlations_df[1]

In [ ]:
strong_correlations_df.drop([0,1],axis=1, inplace=True)

In [ ]:
strong_correlations_df.rename(columns= {2: "correlaton"}, inplace=True)
strong_correlations_df.sort_values(by="correlaton",ascending=False,inplace=True)
strong_correlations_df.set_index("feature_pair", drop=True)

I wanted to see if there were strong correlations in the features, so I could remove some.<br />
With 400 or so features and something like 600 data points, I thought it would be good to remove redundant features.<br />
I found strongly correlated features, but my performance was good.<br /> I learned that Random Forests can be
tolerant of correlated features, so removing them seems uneccesary.

In [ ]:
#strong_correlations_df

If the test accuracy is a lot lower than the train accuracy, it could be a sign of overfitting

In [ ]:
pred_test=RF.predict(X_test)  
print("Train accuracy = " + str(RF.score(X_train,y_train)))
print("Test accuracy = " + str(RF.score(X_test,y_test)))

In [ ]:
# What about AUROC?
prob_y = RF.predict_proba(X)
prob_y
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)


ROC Area Under Curve is a good way of measuring how well the binary classifier separates the two classes, even if the classes are unbalanced

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
print("Train ROC AUC = " + str(roc_auc_score(y_train, prob_y_train[:,1])))
print("Test ROC AUC = " + str(roc_auc_score(y_test, prob_y_test[:,1])))

In [ ]:
estimator = RF.estimators_[0]

In [ ]:
estimator

In [ ]:
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = ('near-Starbucks','not-near'),
                rounded = True, proportion = True, 
                precision = 2, filled = True)

Display a tree<br />
Note: this is only one tree <br />
Random Forest takes a vote from many trees <br />
It is possible however, to use only one tree (with some loss of accuracy),<br />
and interpret how the classifier is working

In [ ]:
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

Can we do better using recursive feature elimination?

In [ ]:
from sklearn.feature_selection import RFECV
rf_limited_features = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)  
rfecv = RFECV(estimator=rf_limited_features, step=1, cv=2, scoring='roc_auc', verbose=2) 
selector=rfecv.fit(X_train, y_train)

In [ ]:
#sk.metrics.SCORERS.keys()#this shows the scoring metric choices

In [ ]:
reduced_features_preds = selector.predict(X_train)

In [ ]:
selector.score(X_train,y_train)

In [ ]:
selector_score = selector.score(X_test,y_test)
selector_score

In [ ]:
selector.n_features_

In [ ]:
selector

In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

Selector picked 140 features, but that looks like a noisy spike<br />
Half that (~70) looks like it would do fine, but there is no real loss by keeping extra features<br />
Random Forest model does fairly well with extra features

Give svm a try

In [ ]:
from sklearn import svm
SVM_model = svm.SVC(kernel='rbf', C=1, gamma=1, probability=True) 
SVM_model.fit(X_train, y_train)
SVM_predicted = SVM_model.predict(X_test)
SVM_model.score(X_train,y_train)
print("Train accuracy = " + str(SVM_model.score(X_train,y_train)))
SVM_model.score(X_test,y_test)
print("Test accuracy = " + str(SVM_model.score(X_test,y_test)))

# What about AUROC?
svm_prob_y = SVM_model.predict_proba(X)
svm_prob_y
svm_prob_y_train = SVM_model.predict_proba(X_train)
svm_prob_y_test = SVM_model.predict_proba(X_test)

print("SVM Train ROC AUC = " + str(roc_auc_score(y_train, svm_prob_y_train[:,1])))
print("SVM Test ROC AUC = " + str(roc_auc_score(y_test, svm_prob_y_test[:,1])))

In [ ]:
SVM_model = svm.SVC(kernel='linear', C=1, gamma=1, probability=True) 
SVM_model.fit(X_train, y_train)
SVM_predicted = SVM_model.predict(X_test)
SVM_model.score(X_train,y_train)
print("Train accuracy = " + str(SVM_model.score(X_train,y_train)))
SVM_model.score(X_test,y_test)
print("Test accuracy = " + str(SVM_model.score(X_test,y_test)))

# What about AUROC?
svm_prob_y = SVM_model.predict_proba(X)
svm_prob_y
svm_prob_y_train = SVM_model.predict_proba(X_train)
svm_prob_y_test = SVM_model.predict_proba(X_test)

print("SVM Train ROC AUC = " + str(roc_auc_score(y_train, svm_prob_y_train[:,1])))
print("SVM Test ROC AUC = " + str(roc_auc_score(y_test, svm_prob_y_test[:,1])))

In [ ]:
SVM_model = svm.SVC(kernel='poly', C=1, gamma=1, probability=True) 
SVM_model.fit(X_train, y_train)
SVM_predicted = SVM_model.predict(X_test)
SVM_model.score(X_train,y_train)
print("Train accuracy = " + str(SVM_model.score(X_train,y_train)))
SVM_model.score(X_test,y_test)
print("Test accuracy = " + str(SVM_model.score(X_test,y_test)))

# What about AUROC?
svm_prob_y = SVM_model.predict_proba(X)
svm_prob_y
svm_prob_y_train = SVM_model.predict_proba(X_train)
svm_prob_y_test = SVM_model.predict_proba(X_test)

print("SVM Train ROC AUC = " + str(roc_auc_score(y_train, svm_prob_y_train[:,1])))
print("SVM Test ROC AUC = " + str(roc_auc_score(y_test, svm_prob_y_test[:,1])))

In [ ]:
SVM_model = svm.SVC(kernel='sigmoid', C=1, gamma=1, probability=True) 
SVM_model.fit(X_train, y_train)
SVM_predicted = SVM_model.predict(X_test)
SVM_model.score(X_train,y_train)
print("Train accuracy = " + str(SVM_model.score(X_train,y_train)))
SVM_model.score(X_test,y_test)
print("Test accuracy = " + str(SVM_model.score(X_test,y_test)))

# What about AUROC?
svm_prob_y = SVM_model.predict_proba(X)
svm_prob_y
svm_prob_y_train = SVM_model.predict_proba(X_train)
svm_prob_y_test = SVM_model.predict_proba(X_test)

print("SVM Train ROC AUC = " + str(roc_auc_score(y_train, svm_prob_y_train[:,1])))
print("SVM Test ROC AUC = " + str(roc_auc_score(y_test, svm_prob_y_test[:,1])))

Looks like RandomForest and SVM with an rbf kernel do well<br />
Maybe we should try those with smaller feature sets

In [ ]:
#tried dropping my added feature but lets keep it
#X_train.drop(['number_of_venues_within_radius'], axis=1)
#X_test.drop(['number_of_venues_within_radius'], axis=1)

In [ ]:



RF = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
fit = RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)
pred_test=RF.predict(X_test)  
print("Train accuracy = " + str(RF.score(X_train,y_train)))
print("Test accuracy = " + str(RF.score(X_test,y_test)))

# What about AUROC?
prob_y = RF.predict_proba(X)
prob_y
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)
print("Train ROC AUC = " + str(roc_auc_score(y_train, prob_y_train[:,1])))
print("Test ROC AUC = " + str(roc_auc_score(y_test, prob_y_test[:,1])))
num_features = fit.n_features_
print("Num Features: " + str(num_features))  

In [ ]:
X.columns

In [ ]:
pdfid

In [ ]:
pdfid.size

In [ ]:
#pdfid.loc['Paintball Field']['Feature Importance']

In [ ]:
pdfid.shape

In [ ]:
##### validate #####

Let's try this model on new data

In [ ]:
#starbucks heavy area in Illinois
#41.916715, -87.989170
#41.843114, -87.779305
validate_llat = 41.843114
validate_llon = -87.989170
validate_ulat = 41.916715
validate_ulon = -87.779305
generate_points(validate_llat, validate_llon, validate_ulat, validate_ulon)
validation_points = points

In [ ]:
validation_points=pd.DataFrame(validation_points)
len(validation_points)

In [ ]:
validation_points = validation_points.sample(frac=1).reset_index(drop=True)
validation_points.head()
validation_venues = getNearbyVenues(latitudes = validation_points[0],
                              longitudes =  validation_points[1]
                               )

In [ ]:
validation_venues.tail()
validation_venues[0:3]

In [ ]:
validation_venues['Venue'].value_counts()

In [ ]:
# one hot encoding
validation_onehot = pd.get_dummies(validation_venues[['Venue Category']], prefix="", prefix_sep="")


In [ ]:
validation_onehot.head()

In [ ]:
# add neighborhood column back to dataframe
validation_onehot['point_name'] = validation_venues['Point Name'] 
validation_onehot['point_lat'] = validation_venues['Neighborhood Latitude'] 
validation_onehot['point_lon'] = validation_venues['Neighborhood Longitude'] 
validation_onehot['isStarbucks'] = (validation_venues['Venue'] == "Starbucks")# should do a better fuzzy match
# move neighborhood column to the first column
#validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])


In [ ]:
# move neighborhood column to the first column
validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])

In [ ]:
len(validation_fixed_columns)

In [ ]:
validation_fixed_columns

In [ ]:
validation_onehot.columns

In [ ]:
len(validation_fixed_columns)

In [ ]:
validation_fixed_columns

In [ ]:
len(validation_onehot.columns)

In [ ]:
validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])

In [ ]:
validation_onehot = validation_onehot[validation_fixed_columns]

In [ ]:
validation_onehot.isStarbucks = validation_onehot.isStarbucks.astype(int)

In [ ]:
validation_onehot.head()
validation_onehot['isStarbucks'].value_counts()

In [ ]:
validation_venues.head()
len(validation_onehot)

In [ ]:
validation_points_near_starbucks = validation_onehot[validation_onehot.isStarbucks > 0]["point_name"]
if eliminate_starbucks_venue_rows == True:
    #keep the isStarbucks flag but throw out other venue info
    ##points_near_starbucks = onehot[onehot.isStarbucks > 0]["point_name"]
    validation_indexes_to_remember = validation_onehot[validation_onehot.isStarbucks > 0].index
    validation_points_near_starbucks

In [ ]:
validation_points_near_starbucks_df = pd.DataFrame(validation_points_near_starbucks)
validation_points_near_starbucks_df.head()

In [ ]:
for point_name in validation_points_near_starbucks_df.point_name:
    validation_onehot.loc[validation_onehot.point_name == point_name,"isStarbucks"] = 1 	
	

In [ ]:
len(validation_onehot)

In [ ]:
if eliminate_starbucks_venue_rows == True:
    for ind in validation_indexes_to_remember:
        validation_onehot = validation_onehot.drop([ind], axis=0)	
		

In [ ]:
len(validation_onehot)	
len(validation_onehot.columns)	
validation_onehot['number_of_venues_within_radius'] = 1
len(validation_onehot.columns)

In [ ]:
# create the new feature 'number_of_venues_within_radius'
# have to combine means and counts
validation_grouper = validation_onehot.groupby('point_name').mean().reset_index()
validation_counter = validation_onehot.groupby('point_name').count().reset_index()
validation_grouped = validation_grouper
validation_grouped.iloc[:,-1] = validation_counter.iloc[:,-1]
validation_max_of_venue_count = validation_grouped.iloc[:,-1].max()
# normalize the new feature
validation_grouped.iloc[:,-1] = validation_grouped.iloc[:,-1]/validation_max_of_venue_count
validation_grouped.iloc[:,-1].describe()



In [ ]:
verbose = False
num_top_venues = 5
count = 0
for point in validation_grouped['point_name']:
    validation_temp = validation_grouped[validation_grouped['point_name'] == point].T.reset_index()
    validation_temp.columns = ['venue','freq']
    validation_temp = validation_temp.iloc[2:]
    validation_temp['freq'] = validation_temp['freq'].astype(float)
    validation_temp = validation_temp.round({'freq': 2})
    if verbose == True:
        print("----"+point+"----")
        print(validation_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')
    else:
        if count % 100 == 0:
            print("----"+point+"----")
            print(validation_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
            print('\n')
    count = count + 1


In [ ]:
validation_grouped.head()
validation_y = validation_grouped['isStarbucks'] > 0
validation_y = pd.DataFrame(validation_y)
validation_y = validation_y.isStarbucks.astype(int)
print(validation_y.value_counts())
validation_X = validation_grouped
print(validation_X.shape)
validation_X.drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)
validation_X.columns

In [ ]:
validation_X['random'] = np.random.random(size=len(validation_X))
print(validation_X.columns)
print(validation_X.shape)
validation_pdy=pd.DataFrame(validation_y)
validation_pdy['isStarbucks'].value_counts()

In [ ]:
#yikes - my features don't match
X.head()

In [ ]:
validation_X.head()

In [ ]:
#Let's see if we can fit this square peg into that round hole
matchup_validation_X = []
matchup_validation_X = pd.DataFrame(matchup_validation_X)
#matchup_validation_X.rename(columns=X.columns)
#matchup_validation_X = X

In [ ]:
len(validation_X)

In [ ]:
matchup_validation_X = pd.DataFrame(0, index=np.arange(len(validation_X)), columns=X.columns)
matchup_validation_X.head()

In [ ]:
matchup_validation_X.tail()

In [ ]:
matchup_validation_X.describe()

In [ ]:
for val_col in validation_X.columns:
    matchup_validation_X[val_col] = validation_X[val_col]

In [ ]:
matchup_validation_X.describe()

In [ ]:
X.describe()

In [ ]:
matchup_validation_X.shape

Looks like we have some extra columns at the end
Would be nice to have incorporated those extra rows at the beginning
For now, we just throw those extra columns out

In [ ]:
while len(matchup_validation_X.columns) > len(X.columns):
    matchup_validation_X.drop(matchup_validation_X.columns[len(matchup_validation_X.columns)-1], axis=1, inplace=True)


In [ ]:
matchup_validation_X.shape

In [ ]:
pd.DataFrame(X).shape

In [ ]:
matchup_validation_X.columns

In [ ]:
X.columns

In [ ]:
validation_X = matchup_validation_X

In [ ]:
validation_preds = RF.predict(validation_X)  
validation_score = RF.score(validation_X,validation_y)
print("Validation accuracy = " + str(validation_score))

# What about AUROC?
validation_prob_y = RF.predict_proba(validation_X)
validation_prob_y = RF.predict_proba(validation_X)

print("Validation ROC AUC = " + str(roc_auc_score(validation_y, validation_prob_y[:,1])))

###############
##############
#### validate end ###
################	

In [ ]:
##### end validate #####

In [ ]:
# drop features with importance below cutoff
cutoff = selector.n_features_
#for feature, feature_imp in pdfid[cutoff:pdfid.size].index,pdfid[cutoff:pdfid.size]['Feature Importance']:
for feature in pdfid[cutoff:pdfid.size].index: 
    print("removing feature: " + str(feature) + " with importance = " + str(pdfid.loc[feature]['Feature Importance']))
    X_train.drop(columns=[feature], inplace=True)
    X_test.drop(columns=[feature], inplace=True)    


In [ ]:
len(X_train.columns)

In [ ]:
X_train.columns

In [ ]:
RF = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
fit = RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)
pred_test=RF.predict(X_test)  
print("Train accuracy = " + str(RF.score(X_train,y_train)))
print("Test accuracy = " + str(RF.score(X_test,y_test)))

# What about AUROC?
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)
print("Train ROC AUC = " + str(roc_auc_score(y_train, prob_y_train[:,1])))
print("Test ROC AUC = " + str(roc_auc_score(y_test, prob_y_test[:,1])))
num_features = fit.n_features_
print("Num Features: " + str(num_features)) 


In [ ]:
feature_importances = RF.feature_importances_
feature_importance_info = []
for i in range(0,num_features):
    if feature_importances[i] >= 0:
        feature_importance_info.append((feature_importances[i],X_train.columns[i]))
pdfid = pd.DataFrame(feature_importance_info)
pdfid.rename(columns={0: "Feature Importance", 1: "Feature"}, inplace=True)
pdfid.sort_values(by="Feature Importance",ascending=False,inplace=True)  
pdfid.set_index('Feature',inplace=True)
pdfid

In [ ]:
pdfid.shape

In [ ]:
X_train.shape

Let's try this model on new data

In [ ]:
#starbucks heavy area in Illinois
#41.916715, -87.989170
#41.843114, -87.779305
validate_llat = 41.843114
validate_llon = -87.989170
validate_ulat = 41.916715
validate_ulon = -87.779305
generate_points(validate_llat, validate_llon, validate_ulat, validate_ulon)
validation_points = points

In [ ]:
validation_points=pd.DataFrame(validation_points)

In [ ]:
len(validation_points)

In [ ]:
################
###############
#### validate ####


validation_points = validation_points.sample(frac=1).reset_index(drop=True)
validation_points.head()

validation_venues = getNearbyVenues(latitudes = validation_points[0],
                              longitudes =  validation_points[1]
                               )

							   



In [ ]:
validation_venues.tail()
validation_venues[0:3]

In [ ]:

validation_venues['Venue'].value_counts()

In [ ]:
# one hot encoding
validation_onehot = pd.get_dummies(validation_venues[['Venue Category']], prefix="", prefix_sep="")


In [ ]:
validation_onehot.head()

In [ ]:




# add neighborhood column back to dataframe
validation_onehot['point_name'] = validation_venues['Point Name'] 
validation_onehot['point_lat'] = validation_venues['Neighborhood Latitude'] 
validation_onehot['point_lon'] = validation_venues['Neighborhood Longitude'] 
validation_onehot['isStarbucks'] = (validation_venues['Venue'] == "Starbucks")# should do a better fuzzy match
# move neighborhood column to the first column
#validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])


In [ ]:
# move neighborhood column to the first column
validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])


In [ ]:
len(validation_fixed_columns)

In [ ]:
validation_fixed_columns

In [ ]:
validation_onehot.columns

In [ ]:
len(validation_fixed_columns)

In [ ]:
validation_fixed_columns

In [ ]:
len(validation_onehot.columns)

In [ ]:
validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])


In [ ]:
validation_onehot = validation_onehot[validation_fixed_columns]

In [ ]:
validation_onehot.isStarbucks = validation_onehot.isStarbucks.astype(int)

In [ ]:





validation_onehot.head()
validation_onehot['isStarbucks'].value_counts()

validation_venues.head()
len(validation_onehot)


validation_points_near_starbucks = validation_onehot[validation_onehot.isStarbucks > 0]["point_name"]
if eliminate_starbucks_venue_rows == True:
    #keep the isStarbucks flag but throw out other venue info
    ##points_near_starbucks = onehot[onehot.isStarbucks > 0]["point_name"]
    validation_indexes_to_remember = validation_onehot[validation_onehot.isStarbucks > 0].index
    validation_points_near_starbucks
	
	
validation_points_near_starbucks_df = pd.DataFrame(validation_points_near_starbucks)
validation_points_near_starbucks_df.head()

for point_name in validation_points_near_starbucks_df.point_name:
    validation_onehot.loc[validation_onehot.point_name == point_name,"isStarbucks"] = 1 	
	
len(validation_onehot)	
	
if eliminate_starbucks_venue_rows == True:
    for ind in validation_indexes_to_remember:
        validation_onehot = validation_onehot.drop([ind], axis=0)	
		
len(validation_onehot)	
len(validation_onehot.columns)	
validation_onehot['number_of_venues_within_radius'] = 1
len(validation_onehot.columns)

# create the new feature 'number_of_venues_within_radius'
# have to combine means and counts
validation_grouper = validation_onehot.groupby('point_name').mean().reset_index()
validation_counter = validation_onehot.groupby('point_name').count().reset_index()
validation_grouped = validation_grouper
validation_grouped.iloc[:,-1] = validation_counter.iloc[:,-1]
validation_max_of_venue_count = validation_grouped.iloc[:,-1].max()
# normalize the new feature
validation_grouped.iloc[:,-1] = validation_grouped.iloc[:,-1]/validation_max_of_venue_count
validation_grouped.iloc[:,-1].describe()






In [ ]:

verbose = False
num_top_venues = 5
count = 0
for point in validation_grouped['point_name']:
    validation_temp = validation_grouped[validation_grouped['point_name'] == point].T.reset_index()
    validation_temp.columns = ['venue','freq']
    validation_temp = validation_temp.iloc[2:]
    validation_temp['freq'] = validation_temp['freq'].astype(float)
    validation_temp = validation_temp.round({'freq': 2})
    if verbose == True:
        print("----"+point+"----")
        print(validation_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')
    else:
        if count % 100 == 0:
            print("----"+point+"----")
            print(validation_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
            print('\n')
    count = count + 1

validation_grouped.head()



validation_y = validation_grouped['isStarbucks'] > 0
validation_y = pd.DataFrame(validation_y)
validation_y = validation_y.isStarbucks.astype(int)
print(validation_y.value_counts())
validation_X = validation_grouped
print(validation_X.shape)
validation_X.drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)



validation_X.columns


In [ ]:

validation_X['random'] = np.random.random(size=len(validation_X))

print(validation_X.columns)
print(validation_X.shape)

validation_pdy=pd.DataFrame(validation_y)
validation_pdy['isStarbucks'].value_counts()


In [ ]:
#yikes - my features don't match
X.head()


In [ ]:
validation_X.head()

In [ ]:
#Let's see if we can fit this square peg into that round hole
matchup_validation_X = []
matchup_validation_X = pd.DataFrame(matchup_validation_X)
#matchup_validation_X.rename(columns=X.columns)
#matchup_validation_X = X

In [ ]:
len(validation_X)

In [ ]:
matchup_validation_X = pd.DataFrame(0, index=np.arange(len(validation_X)), columns=X.columns)
matchup_validation_X.head()


In [ ]:
matchup_validation_X.tail()

In [ ]:
matchup_validation_X.describe()

In [ ]:
for val_col in validation_X.columns:
    matchup_validation_X[val_col] = validation_X[val_col]

In [ ]:
matchup_validation_X.describe()

In [ ]:
X.describe()

In [ ]:
matchup_validation_X.shape

Looks like we have some extra columns at the end<br />
Would be nice to have incorporated those extra rows at the beginning<br />
For now, we just throw those extra columns out

In [ ]:
while len(matchup_validation_X.columns) > len(X.columns):
    matchup_validation_X.drop(matchup_validation_X.columns[len(matchup_validation_X.columns)-1], axis=1, inplace=True)

In [ ]:
matchup_validation_X.shape

In [ ]:
pd.DataFrame(X).shape

In [ ]:
matchup_validation_X.columns

In [ ]:
X.columns

I think I got the new data into the right feature form

In [ ]:

#validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])
#validation_onehot = validation_onehot[validation_fixed_columns]

In [ ]:
validation_X = matchup_validation_X

In [ ]:

validation_preds = RF.predict(validation_X)  
validation_score = RF.score(validation_X,validation_y)
print("Validation accuracy = " + str(validation_score))

# What about AUROC?
validation_prob_y = RF.predict_proba(validation_X)
validation_prob_y = RF.predict_proba(validation_X)

print("Validation ROC AUC = " + str(roc_auc_score(validation_y, validation_prob_y[:,1])))

###############
##############
#### validate end ###
################

In [ ]:
# drop features with importance below cutoff
cutoff = 50
#for feature, feature_imp in pdfid[cutoff:pdfid.size].index,pdfid[cutoff:pdfid.size]['Feature Importance']:
for feature in pdfid[cutoff:pdfid.size].index: 
    print("removing feature: " + str(feature) + " with importance = " + str(pdfid.loc[feature]['Feature Importance']))
    X_train.drop(columns=[feature], inplace=True)
    X_test.drop(columns=[feature], inplace=True)   

In [ ]:
RF = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
fit = RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)
pred_test=RF.predict(X_test)  
print("Train accuracy = " + str(RF.score(X_train,y_train)))
print("Test accuracy = " + str(RF.score(X_test,y_test)))

# What about AUROC?
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)
print("Train ROC AUC = " + str(roc_auc_score(y_train, prob_y_train[:,1])))
print("Test ROC AUC = " + str(roc_auc_score(y_test, prob_y_test[:,1])))
num_features = fit.n_features_
print("Num Features: " + str(num_features)) 

In [ ]:
feature_importances = RF.feature_importances_
feature_importance_info = []
for i in range(0,num_features):
    if feature_importances[i] >= 0:
        feature_importance_info.append((feature_importances[i],X_train.columns[i]))
pdfid = pd.DataFrame(feature_importance_info)
pdfid.rename(columns={0: "Feature Importance", 1: "Feature"}, inplace=True)
pdfid.sort_values(by="Feature Importance",ascending=False,inplace=True)  
pdfid.set_index('Feature',inplace=True)
pdfid


Now let's set the cutoff to the feature importance level where the selector plot first hits<br />
the high plateau

In [ ]:
# drop features with importance below cutoff
cutoff = 20
#for feature, feature_imp in pdfid[cutoff:pdfid.size].index,pdfid[cutoff:pdfid.size]['Feature Importance']:
for feature in pdfid[cutoff:pdfid.size].index: 
    print("removing feature: " + str(feature) + " with importance = " + str(pdfid.loc[feature]['Feature Importance']))
    X_train.drop(columns=[feature], inplace=True)
    X_test.drop(columns=[feature], inplace=True)   

In [ ]:
RF = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
fit = RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)
pred_test=RF.predict(X_test)  
print("Train accuracy = " + str(RF.score(X_train,y_train)))
print("Test accuracy = " + str(RF.score(X_test,y_test)))

# What about AUROC?
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)
print("Train ROC AUC = " + str(roc_auc_score(y_train, prob_y_train[:,1])))
print("Test ROC AUC = " + str(roc_auc_score(y_test, prob_y_test[:,1])))
num_features = fit.n_features_
print("Num Features: " + str(num_features)) 

In [ ]:
feature_importances = RF.feature_importances_
feature_importance_info = []
for i in range(0,num_features):
    if feature_importances[i] >= 0:
        feature_importance_info.append((feature_importances[i],X_train.columns[i]))
pdfid = pd.DataFrame(feature_importance_info)
pdfid.rename(columns={0: "Feature Importance", 1: "Feature"}, inplace=True)
pdfid.sort_values(by="Feature Importance",ascending=False,inplace=True)  
pdfid.set_index('Feature',inplace=True)
pdfid

Now let's set the cutoff to the feature importance level of the random feature

In [ ]:
pdfid.loc['random']
#for i in range(0,len(pdfid)):
#    xx = pdfid.iloc[i]
#    print(xx['Feature'])

In [ ]:
# drop features with importance below cutoff
cutoff = 7
#for feature, feature_imp in pdfid[cutoff:pdfid.size].index,pdfid[cutoff:pdfid.size]['Feature Importance']:
for feature in pdfid[cutoff:pdfid.size].index: 
    print("removing feature: " + str(feature) + " with importance = " + str(pdfid.loc[feature]['Feature Importance']))
    X_train.drop(columns=[feature], inplace=True)
    X_test.drop(columns=[feature], inplace=True)   

In [ ]:
RF = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
fit = RF.fit(X_train, y_train)  
RF.predict(X_train)  
RF.score(X_train,y_train)
pred_test=RF.predict(X_test)  
print("Train accuracy = " + str(RF.score(X_train,y_train)))
print("Test accuracy = " + str(RF.score(X_test,y_test)))

# What about AUROC?
prob_y_train = RF.predict_proba(X_train)
prob_y_test = RF.predict_proba(X_test)
print("Train ROC AUC = " + str(roc_auc_score(y_train, prob_y_train[:,1])))
print("Test ROC AUC = " + str(roc_auc_score(y_test, prob_y_test[:,1])))
num_features = fit.n_features_
print("Num Features: " + str(num_features)) 

In [ ]:
feature_importances = RF.feature_importances_
feature_importance_info = []
for i in range(0,num_features):
    if feature_importances[i] >= 0:
        feature_importance_info.append((feature_importances[i],X_train.columns[i]))
pdfid = pd.DataFrame(feature_importance_info)
pdfid.rename(columns={0: "Feature Importance", 1: "Feature"}, inplace=True)
pdfid.sort_values(by="Feature Importance",ascending=False,inplace=True)  
pdfid.set_index('Feature',inplace=True)
pdfid